# Rock Melody Transformer Training Dataset Maker (ver. 2.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone --depth 1 https://github.com/asigalov61/tegridy-tools

In [ ]:
#@title Import all needed modules

print('Loading core modules. Please wait...')

import os
import copy
import math
import statistics
import random

from joblib import Parallel, delayed, parallel_config

from collections import Counter

from tqdm import tqdm

from google.colab import files

print('Creating IO dirs...')

if not os.path.exists('/content/Dataset'):
  os.makedirs('/content/Dataset')

if not os.path.exists('/content/INTS'):
  os.makedirs('/content/INTS')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD MIDI DATASET)

In [ ]:
# @title Download and untar LAKH clean_midi MIDI subset
%cd /content/Dataset/
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar -xvf clean_midi.tar.gz
!rm clean_midi.tar.gz
%cd /content/

# (FILE LIST)

In [ ]:
#@title Save file list
###########

print('=' * 70)
print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"

filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    for file in filenames:
        if file.endswith(('.mid', '.midi', '.kar')):
            filez.append(os.path.join(dirpath, file))
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

else:
  print('Randomizing file list...')
  random.shuffle(filez)
  print('=' * 70)

  TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/filez')
  print('=' * 70)

In [ ]:
#@title Load file list

print('=' * 70)
filez = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/filez')
print('Done!')
print('=' * 70)

# (PROCESS)

In [ ]:
# @title Load TMIDIX MIDI Processor

print('=' * 70)
print('Loading TMIDIX MIDI Processor...')

def TMIDIX_MIDI_Processor(midi_file):

    try:

        fn = os.path.basename(midi_file)

        #=======================================================
        # START PROCESSING

        #===============================================================================
        # Raw single-track ms score

        raw_score = TMIDIX.midi2single_track_ms_score(midi_file)

        #===============================================================================
        # Enhanced score notes

        escore_notes = TMIDIX.advanced_score_processor(raw_score, return_enhanced_score_notes=True)[0]

        escore_notes = [e for e in escore_notes if e[6] < 80 or e[6] == 128]

        # checking number of instruments in a composition
        instruments_list = list(set([y[3] for y in escore_notes]))

        if len(escore_notes) > 0 and (9 in instruments_list):

            #=======================================================
            # PRE-PROCESSING

            #===============================================================================
            # Augmented enhanced score notes

            escore_notes = TMIDIX.augment_enhanced_score_notes(escore_notes)

            #===============================================================================
            # Chordified score
            dcscore = TMIDIX.chordify_score([1000, escore_notes])

            #===============================================================================

            output_score = []

            pc = dcscore[0]
            pc.sort(key=lambda x: x[4])

            start = False

            for i, c in enumerate(dcscore[:-1]):

              c.sort(key=lambda x: x[4], reverse=True)
              cchans = sorted(set([x[3] for x in c]))

              nc = dcscore[i+1]
              nc.sort(key=lambda x: x[4], reverse=True)
              ncchans = sorted(set([x[3] for x in nc]))

              if 9 in cchans and len(cchans) > 1 and not start:
                    start = True
                    pc = c
                    pc.sort(key=lambda x: x[4])

              dtime = max(0, min(255, (c[0][1] - pc[0][1])))

              if start:

                if 9 in ncchans:
                  ncdrums = 1
                else:
                  ncdrums = 0

                if ncchans == [9]:
                  nctone = 24
                else:
                  nctone = (12 * ncdrums) + (nc[0][4] % 12)

                output_score.append(nctone+2056)

                output_score.append(dtime)

                for e in c:

                    dur = max(1, min(255, e[2]))

                    if e[3] != 9:
                      chan = e[6] // 8
                    else:
                      chan = 11

                    ptc = max(1, min(127, e[4]))

                    cha_ptc = (128 * chan) + ptc

                    velocity = max(8, min(127, e[5]))
                    vel = round(velocity / 15)-1

                    output_score.extend([dur+256, cha_ptc+512, vel+2048])

                pc = c

              if len(output_score) > 8192:
                break

            #===============================================================================

            # TOTAL DICT SIZE 2081

            #===============================================================================

            output_score = output_score[:8193]

            if output_score:
              return output_score

            else:
              return None

        else:
          return None

    except Exception as e:
        print('=' * 70)
        print('ERROR!!!')
        print('File name:', midi_file)
        print('Error:', e)
        print('=' * 70)
        return None

print('Done!')
print('=' * 70)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

NUMBER_OF_PARALLEL_JOBS = 16 # Number of parallel jobs
NUMBER_OF_FILES_PER_ITERATION = 16 # Number of files to queue for each parallel iteration
SAVE_EVERY_NUMBER_OF_ITERATIONS = 160 # Save every 2560 files

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

melody_chords_f = []

files_count = 0
good_files_count = 0

print('Processing MIDI files. Please wait...')
print('=' * 70)

for i in tqdm(range(0, len(filez), NUMBER_OF_FILES_PER_ITERATION)):

  with parallel_config(backend='threading', n_jobs=NUMBER_OF_PARALLEL_JOBS, verbose = 0):
    output = Parallel(n_jobs=NUMBER_OF_PARALLEL_JOBS, verbose=0)(delayed(TMIDIX_MIDI_Processor)(f) for f in filez[i:i+NUMBER_OF_FILES_PER_ITERATION])

  for o in output:

      if o is not None:
          melody_chords_f.append(o)

  # Saving every 2560 processed files
  if i % (SAVE_EVERY_NUMBER_OF_ITERATIONS * NUMBER_OF_FILES_PER_ITERATION) == 0 and i != 0:
      print('SAVING !!!')
      print('=' * 70)
      good_files_count += len(melody_chords_f)
      print('Saving processed files...')
      print('=' * 70)
      print('Data check:', min(melody_chords_f[0]), '===', max(melody_chords_f[0]), '===', len(list(set(melody_chords_f[0]))), '===', len(melody_chords_f[0]))
      print('=' * 70)
      print('Processed so far:', good_files_count, 'out of', i, '===', good_files_count / i, 'good files ratio')
      print('=' * 70)
      count = str(i)
      TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/INTS/RMT_INTs_'+count)
      melody_chords_f = []
      print('=' * 70)

print('SAVING !!!')
print('=' * 70)
good_files_count += len(melody_chords_f)
print('Saving processed files...')
print('=' * 70)
print('Data check:', min(melody_chords_f[0]), '===', max(melody_chords_f[0]), '===', len(list(set(melody_chords_f[0]))), '===', len(melody_chords_f[0]))
print('=' * 70)
print('Processed so far:', good_files_count, 'out of', i, '===', good_files_count / i, 'good files ratio')
print('=' * 70)
count = str(i)
TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/INTS/RMT_INTs_'+count)
print('=' * 70)

# (TEST INTS)

In [ ]:
#@title Test training INTs

train_data1 = random.choice(melody_chords_f)

print('=' * 70)
print('Seq len:', len(train_data1))
print('Sample INTs', train_data1[:15])
print('=' * 70)

out = train_data1

if len(out) != 0:

    song = out
    song_f = []

    time = 0
    dur = 32
    channel = 0
    pitch = 60
    vel = 90

    pmel_tone = 24

    patches = [0, 10, 19, 24, 35, 40, 52, 56, 65, 9, 73, 46, 0, 0, 0, 0]

    for ss in song:

        if 0 <= ss < 256:

            time += ss * 16

        if 256 < ss < 512:

            dur =  (ss-256) * 16

        if 512 < ss < 2048:

            chan = (ss-512) // 128

            if chan == 11:
              channel = 9
            else:
              if chan > 8:
                channel = chan + 1
              else:
                channel = chan

            if channel == 9:
              patch = 128
            else:
              patch = channel * 8

            pitch = (ss-512) % 128

        if 2048 < ss < 2056:
            vel = ((ss - 2048)+1) * 15

            song_f.append(['note', time, dur, channel, pitch, vel, patch])

        if 2056 <= ss < 2080:

            mel_tone = (ss-2056) % 12
            mel_tone_type = (ss-2056) // 12 # 0-2

            if pmel_tone < 24:
                song_f.append(['note', time, 32, 11, 72+pmel_tone, 115, 46])

            pmel_tone = mel_tone

detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Rock Melody Transformer',
                                                        output_file_name = '/content/Rock-Melody-Transformer-Composition',
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=patches
                                                        )
print('=' * 70)

# (ZIP AND DOWNLOAD INTS)

In [ ]:
#@title Zip and download training INTs

print('=' * 70)

try:
    os.remove('Rock_Melody_Transformer_INTs.zip')
except OSError:
    pass

print('Zipping... Please wait...')
print('=' * 70)

%cd /content/INTS/
!zip Rock_Melody_Transformer_INTs.zip *.pickle
%cd /content/

print('=' * 70)
print('Done!')
print('=' * 70)

print('Downloading final zip file...')
print('=' * 70)

files.download('/content/INTS/Rock_Melody_Transformer_INTs.zip')

print('Done!')
print('=' * 70)

# Congrats! You did it! :)